In [1]:
%config IPCompleter.greedy = True
%autosave 60

Autosaving every 60 seconds


## 1. NLP

### Cargar archivo

In [2]:
import pandas as pd
import os
ruta_archivo = os.path.join("imdb_dataset.csv")
df_criticas = pd.read_csv(ruta_archivo, encoding='iso-8859-2').sample(2000, replace=False)
#df_criticas = pd.read_csv(ruta_archivo, encoding='iso-8859-2')
df_criticas

,Review,Label
43220,yeah right. Sammo Hung already acted in the ma...,pos
32323,"Loaded with fine actors, I expected much more ...",neg
11811,"Although allegedly autobiographical, this movi...",neg
41119,My personal opinion is that this movie had no ...,pos
280,Is it possible for a movie to get any worse th...,neg
...,...,...
34950,The same night that I watched this I also watc...,neg
347,Found this film in a DVD discount rack for $10...,neg
35715,- A film crew is shooting a horror movie in an...,neg
9713,"After spending half an hour examining Rumors, ...",neg


### Palabras de parada

In [3]:
import nltk
import string
# nltk.download('punkt')
# nltk.download('stopwords')
palabras_de_parada_ingles = set(nltk.corpus.stopwords.words('english') + list(string.punctuation) + ['...', '..'])
palabras_de_parada_ingles

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '..',
 '...',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 '

### Normalizar el texto

In [4]:
# LimpiadorTexto es una clase que nosotros creamos.
# Utilizamos la funcion normalizar aquí, en el notebook. y también en la API.
# Por tanto, para no tener código repetido, abstraímos la lógica a una clase externa.
from LimpiadorTexto import LimpiadorTexto
limpiador = LimpiadorTexto(palabras_de_parada_ingles)
df_criticas = df_criticas.apply(limpiador.normalizar_fila, axis=1)
df_criticas

,Review,Label
43220,yeah right sammo hung already acted main role ...,pos
32323,loaded fine actors expected much deceiver deli...,neg
11811,although allegedly autobiographical movie demo...,neg
41119,personal opinion movie real story line first j...,pos
280,possible movie get worse bunch apes wandering ...,neg
...,...,...
34950,night watched also watched scary movie 4 makin...,neg
347,found film dvd discount rack 10 worth camera w...,neg
35715,film crew shooting horror movie old supposedly...,neg
9713,spending half hour examining rumors gay bar lo...,neg


### Obtener el vocabulario del problema

In [5]:
def obtener_vocabulario_problema(df_criticas_normalizadas):
    cuerpo_texto_normalizado = ' '.join(df_criticas_normalizadas['Review'].tolist())
    vocabulario_problema = cuerpo_texto_normalizado.split()
    vocabulario_ordenado = sorted(set(vocabulario_problema))
    return vocabulario_ordenado

def obtener_vocabulario_problema_y_posicion(vocabulario_del_problema_ordenado):
    vocabulario_y_posicion = {}
    for i, token in enumerate(vocabulario_del_problema_ordenado):
        vocabulario_y_posicion[token] = i
    return vocabulario_y_posicion

In [6]:
vocabulario_problema = obtener_vocabulario_problema_y_posicion(obtener_vocabulario_problema(df_criticas))
len(vocabulario_problema)

29876

### One Hot Encoding

In [7]:
import numpy as np

def obtener_one_hot_vector(critica, vocabulario_problema_y_posicion):
    one_hot_vector = np.zeros(len(vocabulario_problema_y_posicion), dtype=int)
    for token in critica.split():
        one_hot_vector[vocabulario_problema_y_posicion[token]] = 1
    return one_hot_vector

In [8]:
one_hots = []
indices = []
for indice, fila in df_criticas.iterrows():
    one_hot = obtener_one_hot_vector(fila['Review'], vocabulario_problema)
    indices.append(indice)
    one_hots.append(one_hot)

## 2. Crear y entrenar el modelo predictivo

In [9]:
from sklearn.model_selection import train_test_split
# se cambia el nombre de la variable 'one_hots' para mas claridad
X = one_hots
Y = df_criticas['Label'].ravel()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

X_train =  np.array(X_train)
X_test =  np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

### Liberar la memoria RAM

In [10]:
import gc
lst = [df_criticas]
del df_criticas, one_hots, X, Y
del lst
gc.collect()

120

## Entrenar diferentes modelos con diferentes parámetros para encontrar el que produzca mejores resultados

Se entrenarán diferentes modelos con diferentes parámetros. Se guardará las métricas de cada modelo en un archivo para que luego puedan ser comparadas, y así determinar el mejor clasificador.

In [11]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import json

def guardar_objeto_json(nombre_objeto, objeto):
    ruta_archivo = os.path.join(nombre_objeto)
    with open(ruta_archivo, 'w') as fp:
        json.dump(objeto, fp)
    
def obtener_metricas(clasificador):
    predicciones = clasificador.predict(X_test)
    metricas = {
        'accuracy': accuracy_score(Y_test, predicciones),
        'precision': precision_score(Y_test, predicciones, pos_label='pos'),
        'recall': recall_score(Y_test, predicciones, pos_label='pos'),
        'f1': f1_score(Y_test, predicciones, pos_label='pos')
    }
    return metricas

### Regresión logística con los parámetros:

- random_state=42
- solver='sag'
- max_iter=300

In [13]:
from sklearn.linear_model import LogisticRegression

'''
clasificador = LogisticRegression(random_state=0, solver='sag', max_iter=300)
clasificador.fit(X_train, Y_train)
metricas = obtener_metricas(clasificador)
metricas['model'] = 'LogisticRegression'
metricas['solver'] = 'sag'
metricas['random_state'] = 42
metricas['max_iter'] = 300
guardar_objeto_json('metricas-reg-log1.txt', metricas)
'''

"\nclasificador = LogisticRegression(random_state=0, solver='sag', max_iter=300)\nclasificador.fit(X_train, Y_train)\nmetricas = obtener_metricas(clasificador)\nmetricas['model'] = 'LogisticRegression'\nmetricas['solver'] = 'sag'\nmetricas['random_state'] = 42\nmetricas['max_iter'] = 300\nguardar_objeto_json('metricas-reg-log1.txt', metricas)\n"

### Regresión logística con los parámetros:

- random_state=42
- solver='saga'
- n_jobs=-1

In [14]:
clasificador = LogisticRegression(random_state=42, solver='saga', max_iter=300, n_jobs=-1)
clasificador.fit(X_train, Y_train)
metricas = obtener_metricas(clasificador)
metricas['model'] = 'LogisticRegression'
metricas['solver'] = 'saga'
metricas['random_state'] = 42
metricas['max_iter'] = 300
guardar_objeto_json('metricas-reg-log2.txt', metricas)

/home/jordanmontt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


NameError: name 'guardar_objeto' is not defined

### Regresión logística con los parámetros:

- random_state=42
- solver='liblinear'

In [14]:
clasificador = LogisticRegression(random_state=42, solver='liblinear')
clasificador.fit(X_train, Y_train)
metricas = obtener_metricas(clasificador)
metricas['model'] = 'LogisticRegression'
metricas['solver'] = 'liblinear'
metricas['random_state'] = 42
guardar_objeto_json('metricas-reg-log3.txt', metricas)

### Regresión logística con los parámetros:

- solver='lbfgs'
- n_jobs=-1

In [14]:
clasificador = LogisticRegression(solver='lbfgs', n_jobs=-1)
clasificador.fit(X_train, Y_train)
metricas = obtener_metricas(clasificador)
metricas['model'] = 'LogisticRegression'
metricas['solver'] = 'lbfgs'
guardar_objeto_json('metricas-reg-log4.txt', metricas)

### Regresión logística con los parámetros:

- solver='lbfgs'
- n_jobs=-1

In [16]:
clasificador = LogisticRegression(solver='newton-cg', n_jobs=-1)
clasificador.fit(X_train, Y_train)
metricas = obtener_metricas(clasificador)
metricas['model'] = 'LogisticRegression'
metricas['solver'] = 'newton-cg'
guardar_objeto_json('metricas-reg-log5.txt', metricas)

### Gradient Boosted Trees con optimización de hiper-parámetros

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK,Trials
from xgboost.sklearn import XGBClassifier

space =  {
    'n_estimators':hp.quniform('n_estimators',100,1000,1),
    'learning_rate':hp.quniform('learning_rate',0.025,0.5,0.025),
    'max_depth':hp.quniform('max_depth',1,13,1),
    'subsample': hp.quniform('subsample',0.5,1,0.05),
    'colsample_bytree':hp.quniform('colsample_bytree',0.5,1,0.05),
    'nthread':6, #cuando sea posible, paralelizar el procesamiento empleando hasta 6 hilos
    'silent':1 # si occuriese un error, continuar con la ejecución
}

def objective(params):
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])  
    clf = XGBClassifier(**params)
    clf.fit(X_train, Y_train)
    accuracy = accuracy_score(Y_test, clf.predict(X_test))
    return {'loss': 1 - accuracy, 'status': STATUS_OK}

trials = Trials()
mejores_parametros = fmin(objective,space, algo=tpe.suggest, trials=trials, max_evals=100)
mejores_parametros['n_estimators'] = int(mejores_parametros['n_estimators'])
mejores_parametros['max_depth'] = int(mejores_parametros['max_depth']) 

clasificador = XGBClassifier(**mejores_parametros)
clasificador.fit(X_train, Y_train)
metricas = obtener_metricas(clasificador)
metricas['model'] = 'GradientBoostedTrees'
metricas['params'] = mejores_parametros
guardar_objeto_json('metricas-GradientBoostedTrees.txt', metricas)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

### Guardar el clasificador como un archivo

In [14]:
import pickle
ruta_archivo_clasificador = os.path.join('clasificador-regresion-logistica.pkl')
archivo_clasificador = open(ruta_archivo_clasificador, 'wb')
pickle.dump(clasificador_reg_log, archivo_clasificador)
archivo_clasificador.close()

### Guardar el vocabulario del problema como un archivo

In [15]:
ruta_archivo_vocabulario = os.path.join('vocabulario-problema.pkl')
archivo_vacabulario = open(ruta_archivo_vocabulario, 'wb')
pickle.dump(vocabulario_problema, archivo_vacabulario)
archivo_vacabulario.close()

### Guardar las palabras de parada como un archivo

In [16]:
ruta_archivo_palabras_parada = os.path.join('palabras-parada.pkl')
archivo_palabras_parada = open(ruta_archivo_palabras_parada, 'wb')
pickle.dump(palabras_de_parada_ingles, archivo_palabras_parada)
archivo_palabras_parada.close()